In [6]:
import torch
from tqdm import tqdm
from heart_desease_dataset import HeartDeseaseDataset
from mp_model import MPModel

In [7]:
dataset_train = HeartDeseaseDataset('dataset', train=True)
train_data_loader = torch.utils.data.DataLoader(dataset_train, batch_size=8, shuffle=True, num_workers=4)

device = 'cpu' if torch.cuda.is_available() else 'cpu'

model = MPModel(in_size=30, n_classes=2)
model.to(device)
model.train()

criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01) 

total_epochs = 100

for epoch in range(total_epochs):
    it_pbar = tqdm(enumerate(train_data_loader), desc='', total=len(train_data_loader))
    for idx, batch in enumerate(train_data_loader):
        
        optimizer.zero_grad()  
        
        x, y = batch

        x = x.to(device)
        y = y.to(device)

        y_pred = model(x)

        loss = criterion(y_pred, y)

        loss.backward()

        optimizer.step()

        it_pbar.set_description(f'Epoch: {epoch}, it {idx}, loss: {loss.item():2.4f}')
        it_pbar.update(1)

Epoch: 45, it 56, loss: 0.2310: 100%|██████████| 57/57 [00:00<00:00, 137.18it/s]


In [ ]:
test_dataset    = HeartDeseaseDataset('dataset', train=False)
test_dataloader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True, num_workers=2)

model.eval()

corret_samples = 0
for batch in tqdm(test_dataloader):
    x, y = batch

    x = x.to(device)
    y = y.to(device)
    with torch.no_grad():
        y_hat = model(x)

    corret_samples += y_hat.argmax(dim=1).eq(y).sum()

print(f'Final Acurancy on test set is: {corret_samples/len(test_dataset)*100}%' )
